In [1]:
# %load_ext autoreload 
# %autoreload 2

from imports import *
import queue_system as qs
from queue_system import *

seed = random.seed(10)
seed_np = np.random.seed(10)

# Aggregation

In [2]:
with open('Q/Q_start.pkl', "rb") as fp:
    Q_start = pickle.load(fp)

In [3]:
weights_dict = {}
# model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max)
# model = StochasticMatching(graph, arrival_rates, queue_max)

len_updates = 100
res = int(len_updates / 50)      


weights_dict['-1'] = {state: np.array([1/K] * K) for state in state_space_ind.keys()}

repeat_weights = 10
learning_rate = 0.1
num_episodes = 10

In [4]:
%%time 
random.seed(6)
np.random.seed(6)
for n in range(repeat_weights):
    weights_dict = {}
    model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max, lambda_l2=0, lambda_l1=0, lr=1e-3, Q=Q_start)
    model = StochasticMatching(graph, arrival_rates, queue_max)
    weights_dict['-1'] =  np.ones([num_states, K]) / K
    
    lr = 2
    
    for i in tqdm(range(len_updates + 1)):
        if (i <= 200) and (i%10 == 0):
            weights = model_agg.train_actor_critic(num_episodes, lr)
            weights_dict[str(i)] = copy.deepcopy(weights) #.copy()
        else:
            weights = model_agg.train_actor_critic(num_episodes, lr)
            
        lr = lr * 0.9
            
    with open('weights/weights_actor_critic2_' + str(n) + '.pkl', 'wb') as output:
        pickle.dump(weights_dict, output) 
        
## stoch match 3 PROVA ENTRAMBI MSE

100%|███████████████████████████████████████| 101/101 [1:21:22<00:00, 48.34s/it]

CPU times: user 18h 28min 21s, sys: 1d 16h 58s, total: 2d 10h 29min 19s
Wall time: 12h 47min 29s
